# Retrieval Augmented Generation - Procurement Contract Analyst -  Palm2 & LangChain

## Installation & Authentication

**Install google-generativeai & langchain**
- Get API KEY from MakerSuite or Google Cloud

In [2]:
GOOGLE_API_KEY=''

In [ ]:
# Install langchain and related libraries
!pip install langchain google-generativeai unstructured

# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform==1.25.0


**(Or) Authenticate & Initialize Google Cloud Project**

In [8]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()



**Define Vertex AI Base Model & Library Classes**

In [9]:

import time

from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")


from pydantic import BaseModel, Extra, root_validator
from typing import Any, Mapping, Optional, List, Dict
from langchain.llms.base import LLM
from langchain.embeddings.base import Embeddings

class _VertexCommon(BaseModel):
    """Wrapper around Vertex AI large language models.

    To use, you should have the
    ``google.cloud.aiplatform.private_preview.language_models`` python package
    installed.
    """
    client: Any = None #: :meta private:
    model_name: str = "text-bison@001"
    """Model name to use."""

    temperature: float = 0.2
    """What sampling temperature to use."""

    top_p: int = 0.8
    """Total probability mass of tokens to consider at each step."""

    top_k: int = 40
    """The number of highest probability tokens to keep for top-k filtering."""

    max_output_tokens: int = 200
    """The maximum number of tokens to generate in the completion."""

    @property
    def _default_params(self) -> Mapping[str, Any]:
        """Get the default parameters for calling Vertex AI API."""
        return {
            "temperature": self.temperature,
            "top_p": self.top_p,
            "top_k": self.top_k,
            "max_output_tokens": self.max_output_tokens
        }

    def _predict(self, prompt: str, stop: Optional[List[str]]) -> str:
        res = self.client.predict(prompt, **self._default_params)
        return self._enforce_stop_words(res.text, stop)

    def _enforce_stop_words(self, text: str, stop: Optional[List[str]]) -> str:
        if stop:
            return enforce_stop_tokens(text, stop)
        return text

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "vertex_ai"

class VertexLLM(_VertexCommon, LLM):
    model_name: str = "text-bison@001"

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the python package exists in environment."""
        try:
            from vertexai.preview.language_models import TextGenerationModel
        except ImportError:
            raise ValueError(
                "Could not import Vertex AI LLM python package. "
            )

        try:
            values["client"] = TextGenerationModel.from_pretrained(values["model_name"])
        except AttributeError:
            raise ValueError(
                "Could not set Vertex Text Model client."
            )

        return values

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call out to Vertex AI's create endpoint.

        Args:
            prompt: The prompt to pass into the model.

        Returns:
            The string generated by the model.
        """
        return self._predict(prompt, stop)


class VertexEmbeddings(Embeddings, BaseModel):
    """Wrapper around Vertex AI large language models embeddings API.

    To use, you should have the
    ``google.cloud.aiplatform.private_preview.language_models`` python package
    installed.
    """
    model_name: str = "textembedding-gecko@001"
    """Model name to use."""

    model: Any
    requests_per_minute: int = 15


    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the python package exists in environment."""
        try:
            from vertexai.preview.language_models import TextEmbeddingModel

        except ImportError:
            raise ValueError(
                "Could not import Vertex AI LLM python package. "
            )

        try:
            values["model"] = TextEmbeddingModel

        except AttributeError:
            raise ValueError(
                "Could not set Vertex Text Model client."
            )

        return values

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
      """Call Vertex LLM embedding endpoint for embedding docs
      Args:
          texts: The list of texts to embed.
      Returns:
          List of embeddings, one for each text.
      """
      self.model = self.model.from_pretrained(self.model_name)

      limiter = rate_limit(self.requests_per_minute)
      results = []
      docs = list(texts)

      while docs:
        # Working in batches of 2 because the API apparently won't let
        # us send more than 2 documents per request to get embeddings.
        head, docs = docs[:2], docs[2:]
        # print(f'Sending embedding request for: {head!r}')
        chunk = self.model.get_embeddings(head)
        results.extend(chunk)
        next(limiter)

      return [r.values for r in results]

    def embed_query(self, text: str) -> List[float]:
      """Call Vertex LLM embedding endpoint for embedding query text.
      Args:
        text: The text to embed.
      Returns:
        Embedding for the text.
      """
      single_result = self.embed_documents([text])
      return single_result[0]

def rate_limit(max_per_minute):
  period = 60 / max_per_minute
  print('Waiting')
  while True:
    before = time.time()
    yield
    after = time.time()
    elapsed = after - before
    sleep_time = max(0, period - elapsed)
    if sleep_time > 0:
      print('.', end='')
      time.sleep(sleep_time)


Vertex AI SDK version: 1.25.0


## Initiatlize Vertex AI

In [10]:
PROJECT_ID = "api-project-503433767370"  # @param {type:"string"}

LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

## Ingest the Contracts to build the context for the LLM

*Load all the Procurement Contract Documents*

In [11]:
from langchain.document_loaders import GCSDirectoryLoader
loader = GCSDirectoryLoader(project_name=PROJECT_ID, bucket="contractunderstandingatticusdataset")
documents = loader.load()

*Split documents into chunks as needed by the token limit of the LLM and let there be an overlap between the chunks*

In [12]:
# split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print(f"# of documents = {len(docs)}")

# of documents = 2150


## Structuring the ingested documents in a vector space using a Vector Database

*Create an embedding vector engine for all the text in the contract documents that have been ingested*

In [13]:
# Embedding has a max of 600 requests per minute so we are within limits
# https://cloud.google.com/vertex-ai/docs/quotas

REQUESTS_PER_MINUTE = 590

embedding = VertexEmbeddings(requests_per_minute=REQUESTS_PER_MINUTE)

embedding

VertexEmbeddings(model_name='textembedding-gecko@001', model=<class 'vertexai.language_models._language_models.TextEmbeddingModel'>, requests_per_minute=590)

*Create a vector store and store the embeddings in the vector store*

In [14]:
# Store docs in local vectorstore as index
!pip install -q chromadb

# it may take a while since API is rate limited
from langchain.vectorstores import Chroma

contracts_vector_db = Chroma.from_documents(docs, embedding)

Waiting
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Obtain handle to the retriever

In [15]:
# Expose index to the retriever
retriever = contracts_vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2})

## Define a Retrieval QA Chain to use retriever

In [16]:
# Create chain to answer questions
from langchain.chains import RetrievalQA

llm = VertexLLM(
    model_name='text-bison-32k',
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

## Leverage LLM to search from retriever

*Example:*

In [17]:
query = "Who all entered into agreement with Sagebrush?"
result = qa({"query": query})
print(result)

Waiting
{'query': 'Who all entered into agreement with Sagebrush?', 'result': ' Allison Transmission Holdings, Inc.', 'source_documents': [Document(page_content='Each party cooperated and participated in the drafting and preparation of this Agreement and the documents referred to herein, and any and all drafts relating thereto exchanged among the parties shall be deemed the work product of all of the parties and may not be construed against any party by reason of its drafting or preparation. Accordingly, any rule of law or any legal decision that would require interpretation of any ambiguities in this Agreement against any party that drafted or prepared it is of no application and is hereby expressly waived by each of the parties hereto, and any controversy over interpretations of this Agreement shall be decided without regards to events of drafting or preparation.\n\n[Signature Pages Follow]   7\n\nIN WITNESS WHEREOF, each of the parties hereto has executed this COOPERATION AGREEMENT 

## Build a Front End

In [44]:
!pip install -q gradio
import gradio as gr
import markdown

def chatbot(inputtext):
    result = qa({"query": inputtext})

    return result['result'], get_public_url(result['source_documents'][0].metadata['source']), result['source_documents'][0].metadata['source']

from google.cloud import storage

def get_public_url(uri):
    """Returns the public URL for a file in Google Cloud Storage."""
    # Split the URI into its components
    components = uri.split("/")

    # Get the bucket name
    bucket_name = components[2]

    # Get the file name
    file_name = components[3]

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    return blob.public_url


print("Launching Gradio")

iface = gr.Interface(fn=chatbot,
                     inputs=[gr.Textbox(label="Query")],
                     examples=["Who are parties to ADMA agreement", "What is the agreement between MICOA & Stratton Cheeseman", "What is the commission % that Stratton Cheeseman will get from MICOA and how much will they get if MICOA's revenues are $100"],
                     title="Contract Analyst",
                     outputs=[gr.Textbox(label="Response"),
                              gr.Textbox(label="URL"),
                              gr.Textbox(label="Cloud Storage URI")],
                     theme=gr.themes.Soft)

iface.launch(share=False)



Launching Gradio


/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:611: UserWarning: Theme should be a class loaded from gradio.themes
  warnings.warn("Theme should be a class loaded from gradio.themes")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>